In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,602 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:14 http://archive.ubuntu.com/ubunt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

search_terms = ['facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

scraped_articles_count = 0
MAX_ARTICLES = 5000  # had a limit in the beginning to see if scraped article was scraped correctly before moving on to all

def scrape_article(article_url, search_term):
    global scraped_articles_count
    driver.get(article_url)

    if "https://www.dr.dk/nyheder/seneste/" in article_url: #variation in the articles. when articles are 'seneste', then one way of scraping. when otherwise, another
        try:
            header = driver.find_element(By.CLASS_NAME, 'dre-title-text').text
        except:
            header = 'no header'

        subheader = ''


        # Extract content for seneste articles
        all_text = []
        try:
            speech_div = driver.find_element(By.CLASS_NAME, 'hydra-latest-news-page-short-news-article__body')
            paragraphs = speech_div.find_elements(By.CLASS_NAME, 'dre-speech')
            for paragraph in paragraphs:
                paragraph_content = paragraph.get_attribute("innerHTML")
                soup = BeautifulSoup(paragraph_content, 'html.parser')
                for a in soup.find_all('a'):
                    href = a.get('href')
                    a.replace_with(f'{a.text} [Link: {href}]')
                all_text.append(soup.get_text())
        except:
            all_text = ['No content']

        content = ' '.join(all_text)

        category = 'no category'

        try:
            date = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop="datePublished"]').get_attribute('content')
        except Exception as e:
            print(f"Error extracting published date: {e}")

    else:
        try:
            header = driver.find_element(By.CLASS_NAME, 'dre-title-text').text
        except:
            header = 'no header'

        try:
            subheader = driver.find_element(By.CLASS_NAME, 'dre-article-title__summary.dre-variables').text
        except:
            subheader = 'no subheader'

        # regular articles
        all_text = []
        try:
            speech_divs = driver.find_elements(By.CLASS_NAME, 'dre-speech')
            for speech_div in speech_divs:
                paragraphs = speech_div.find_elements(By.CLASS_NAME, 'dre-article-body-paragraph.dre-variables')
                for paragraph in paragraphs:
                    paragraph_content = paragraph.get_attribute("innerHTML")
                    soup = BeautifulSoup(paragraph_content, 'html.parser')
                    for a in soup.find_all('a'):
                        href = a.get('href')
                        a.replace_with(f'{a.text} [Link: {href}]')
                    all_text.append(soup.get_text())
        except:
            all_text = ['No content']

        content = ' '.join(all_text)

        try:
            date_element = driver.find_element(By.CLASS_NAME, 'dre-byline__date')
            date = date_element.get_attribute('datetime') if date_element else 'N/A'
        except:
            date = 'no date'

        try:
            category_element = driver.find_element(By.CLASS_NAME, 'dre-article-title-section-label__title')
            category_href = category_element.get_attribute('href')
            category = category_href.split('/')[-1] if category_href else 'N/A'
        except:
            category = 'no category'

    scraped_articles_count += 1
    print(f"Scraped {scraped_articles_count} articles so far...")

    return {
        'URL': article_url,
        'Header': header,
        'Subheader': subheader,
        'Content': content,
        'Date': date,
        'Category': category,
        'Search Term': search_term
    }

all_articles = []
scraped_urls = set()  # no duplicates

for term in search_terms:
    if scraped_articles_count >= MAX_ARTICLES:
        break

    print(f"Processing search term: {term}")

    # sort using both publish time and rleevance to get as many articles as possible
    for sort_type in ['PublishTime', 'Relevance']:
        print(f"Sorting by {sort_type}...")

        base_url = f'https://www.dr.dk/soeg?query={term}&sort={sort_type}'

        driver.get(base_url)

        page_number = 1
        articles_before_click = len(driver.find_elements(By.CSS_SELECTOR, 'ul.dre-teaser-list__list li'))

        while True:
            if scraped_articles_count >= MAX_ARTICLES:
                break

            print(f"Processing page {page_number} for term '{term}' sorted by {sort_type}...")

            try:
                show_more_button = driver.find_element(By.CSS_SELECTOR, 'button.dre-button')
                driver.execute_script("arguments[0].click();", show_more_button)

                time.sleep(5)

                articles_after_click = len(driver.find_elements(By.CSS_SELECTOR, 'ul.dre-teaser-list__list li'))

                print(f"Number of articles on page after click: {articles_after_click}")

                if articles_after_click == articles_before_click:
                    print("No new articles loaded, stopping...")
                    break

                articles_before_click = articles_after_click
                page_number += 1

            except Exception as e:
                print(f"No more articles or button not found: {e}")
                break

        article_urls = [article.get_attribute('href') for article in driver.find_elements(By.CSS_SELECTOR, 'ul.dre-teaser-list__list li a')]

        for article_url in article_urls:
            if scraped_articles_count >= MAX_ARTICLES:
                break

            if article_url not in scraped_urls:
                try:
                    article_data = scrape_article(article_url, term)
                    all_articles.append(article_data)
                    scraped_urls.add(article_url)
                except Exception as e:
                    print(f"Error scraping article {article_url}: {e}")

df = pd.DataFrame(all_articles)
df.to_csv('scraped_articles_DR_all_terms_all.csv', index=False)

driver.quit()

print(df.head())


Processing search term: facial recognition
Sorting by PublishTime...
Processing page 1 for term 'facial recognition' sorted by PublishTime...
No more articles or button not found: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button.dre-button"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b6386bbeb9a <unknown>
#1 0x5b63868a4670 <unknown>
#2 0x5b63868f3c48 <unknown>
#3 0x5b63868f3ee1 <unknown>
#4 0x5b638693a924 <unknown>
#5 0x5b6386918bad <unknown>
#6 0x5b6386937dd6 <unknown>
#7 0x5b6386918923 <unknown>
#8 0x5b63868e66e7 <unknown>
#9 0x5b63868e76de <unknown>
#10 0x5b6386b8866b <unknown>
#11 0x5b6386b8c611 <unknown>
#12 0x5b6386b744e5 <unknown>
#13 0x5b6386b8d192 <unknown>
#14 0x5b6386b596ef <unknown>
#15 0x5b6386bad9d8 <unknown>
#16 0x5b6386badba7 <unknown>
#17 0x5b6386bbd9ec <unknown>
#

<ipython-input-2-826157f4daa5>:91: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(paragraph_content, 'html.parser')


Scraped 9 articles so far...
Scraped 10 articles so far...
Scraped 11 articles so far...
Scraped 12 articles so far...
Scraped 13 articles so far...
Scraped 14 articles so far...
Scraped 15 articles so far...
Processing search term: masseovervågning kamera
Sorting by PublishTime...
Processing page 1 for term 'masseovervågning kamera' sorted by PublishTime...
No more articles or button not found: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button.dre-button"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b6386bbeb9a <unknown>
#1 0x5b63868a4670 <unknown>
#2 0x5b63868f3c48 <unknown>
#3 0x5b63868f3ee1 <unknown>
#4 0x5b638693a924 <unknown>
#5 0x5b6386918bad <unknown>
#6 0x5b6386937dd6 <unknown>
#7 0x5b6386918923 <unknown>
#8 0x5b63868e66e7 <unknown>
#9 0x5b63868e76de <unknown>
#10 0x5b6386b8

In [ ]:
df

,URL,Header,Subheader,Content,Date,Category,Search Term
0,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebooks ansigtsgenkendelse er muligvis ulovlig,Facebook er begyndt at analysere ansigter på e...,"Inden længe vil Facebook spørge dig, om de må ...",2018-04-20T12:00:00+00:00,teknologi,face recognition
1,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebook vil have dig til at sige ‘ja’ til ans...,Facebook genintroducerer kontroversiel ansigts...,Må Facebook have lov til at bruge en teknologi...,2018-04-18T17:55:00+00:00,teknologi,face recognition
2,https://www.dr.dk/nyheder/viden/tech/senator-h...,Senator: Hvor sikker er Apples nye ansigtsgenk...,"Det amerikanske senat vil have svar på, hvor s...","Da Apple tirsdag lancerede sin nye iPhone X, v...",2017-09-14T16:33:00+00:00,N/A,ansigtsscan
3,https://www.dr.dk/nyheder/viden/tech/ny-mobilm...,Ny mobilmode: Nu bliver telefonerne rammeløse,De nye telefoner ligner næsten bare en glaspla...,"Apple præsenterer i morgen den næste iPhone, t...",2017-09-11T18:04:00+00:00,N/A,ansigtsscan
4,https://www.dr.dk/nyheder/viden/tech/opsamling...,Opsamling: Anmeldere tager pænt imod iPhone X,"Der er store roser til Apples nye, hundedyre i...","“9 ud af 10”, “1.000 dollars værd”, “nem at an...",2017-11-06T14:00:00+00:00,N/A,ansigtsscan
5,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet 'råber vagt i gevær': Her er spør...,,I sidste uge fik politiet grønt lys til brug a...,2024-09-14T19:01:00+00:00,indland,masseovervågning ansigt
6,https://www.dr.dk/nyheder/udland/her-er-fire-t...,"Her er fire teknologier, der holder kinesiske ...",,Politifolk har stormet kinesiske gader og angi...,2022-11-29T20:22:00+00:00,udland,masseovervågning ansigt
7,https://www.dr.dk/nyheder/udland/it-gigant-skr...,It-gigant skrotter sin egen teknologi og advar...,Myndigheder bør være påpasselige med at bruge ...,It-giganten IBM har besluttet at droppe udvikl...,2020-06-09T16:06:00+00:00,udland,masseovervågning ansigt
8,https://www.dr.dk/nyheder/viden/tech/wikileaks...,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...,Analyse: Lækkede CIA-dokumenter fortæller om t...,Den 7. marts blev en stor samling dokumenter m...,2017-03-21T12:38:00+00:00,N/A,masseovervågning ansigt
9,https://www.dr.dk/nyheder/politik/sf-vil-have-...,SF vil have kommission til at kulegrave overvå...,Alle partier ønsker mere viden om overvågninge...,SF vil have regeringen til at nedsætte en komm...,2014-04-25T04:20:00+00:00,politik,masseovervågning ansigt


In [ ]:
from google.colab import files

df.to_csv('df_DR_all_terms.csv')
files.download('df_DR_all_terms.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/df_DR_all_terms.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
import pandas as pd

# Step 1: Remove everything after the first 10 characters (YYYY-MM-DD)
df['cleaned_date'] = df['Date'].str[:10]

# Step 2: Convert the cleaned date to a standardized format (YYYY-MM-DD)
df['Date'] = pd.to_datetime(df['cleaned_date'], format='%Y-%m-%d', errors='coerce')

df= df.drop(columns=['cleaned_date'])
df

,URL,Header,Subheader,Content,Date,Category,Search Term
0,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebooks ansigtsgenkendelse er muligvis ulovlig,Facebook er begyndt at analysere ansigter på e...,"Inden længe vil Facebook spørge dig, om de må ...",2018-04-20,teknologi,face recognition
1,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebook vil have dig til at sige ‘ja’ til ans...,Facebook genintroducerer kontroversiel ansigts...,Må Facebook have lov til at bruge en teknologi...,2018-04-18,teknologi,face recognition
2,https://www.dr.dk/nyheder/viden/tech/senator-h...,Senator: Hvor sikker er Apples nye ansigtsgenk...,"Det amerikanske senat vil have svar på, hvor s...","Da Apple tirsdag lancerede sin nye iPhone X, v...",2017-09-14,N/A,ansigtsscan
3,https://www.dr.dk/nyheder/viden/tech/ny-mobilm...,Ny mobilmode: Nu bliver telefonerne rammeløse,De nye telefoner ligner næsten bare en glaspla...,"Apple præsenterer i morgen den næste iPhone, t...",2017-09-11,N/A,ansigtsscan
4,https://www.dr.dk/nyheder/viden/tech/opsamling...,Opsamling: Anmeldere tager pænt imod iPhone X,"Der er store roser til Apples nye, hundedyre i...","“9 ud af 10”, “1.000 dollars værd”, “nem at an...",2017-11-06,N/A,ansigtsscan
5,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet 'råber vagt i gevær': Her er spør...,,I sidste uge fik politiet grønt lys til brug a...,2024-09-14,indland,masseovervågning ansigt
6,https://www.dr.dk/nyheder/udland/her-er-fire-t...,"Her er fire teknologier, der holder kinesiske ...",,Politifolk har stormet kinesiske gader og angi...,2022-11-29,udland,masseovervågning ansigt
7,https://www.dr.dk/nyheder/udland/it-gigant-skr...,It-gigant skrotter sin egen teknologi og advar...,Myndigheder bør være påpasselige med at bruge ...,It-giganten IBM har besluttet at droppe udvikl...,2020-06-09,udland,masseovervågning ansigt
8,https://www.dr.dk/nyheder/viden/tech/wikileaks...,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...,Analyse: Lækkede CIA-dokumenter fortæller om t...,Den 7. marts blev en stor samling dokumenter m...,2017-03-21,N/A,masseovervågning ansigt
9,https://www.dr.dk/nyheder/politik/sf-vil-have-...,SF vil have kommission til at kulegrave overvå...,Alle partier ønsker mere viden om overvågninge...,SF vil have regeringen til at nedsætte en komm...,2014-04-25,politik,masseovervågning ansigt


In [ ]:
df['merged_content'] = df[['Header', 'Subheader', 'Content']].fillna('').agg(' '.join, axis=1)

df = df.dropna(subset=['merged_content'])
df.columns = df.columns.str.lower()

In [ ]:
df

,url,header,subheader,content,date,category,search term,merged_content
0,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebooks ansigtsgenkendelse er muligvis ulovlig,Facebook er begyndt at analysere ansigter på e...,"Inden længe vil Facebook spørge dig, om de må ...",2018-04-20,teknologi,face recognition,Facebooks ansigtsgenkendelse er muligvis ulovl...
1,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebook vil have dig til at sige ‘ja’ til ans...,Facebook genintroducerer kontroversiel ansigts...,Må Facebook have lov til at bruge en teknologi...,2018-04-18,teknologi,face recognition,Facebook vil have dig til at sige ‘ja’ til ans...
2,https://www.dr.dk/nyheder/viden/tech/senator-h...,Senator: Hvor sikker er Apples nye ansigtsgenk...,"Det amerikanske senat vil have svar på, hvor s...","Da Apple tirsdag lancerede sin nye iPhone X, v...",2017-09-14,N/A,ansigtsscan,Senator: Hvor sikker er Apples nye ansigtsgenk...
3,https://www.dr.dk/nyheder/viden/tech/ny-mobilm...,Ny mobilmode: Nu bliver telefonerne rammeløse,De nye telefoner ligner næsten bare en glaspla...,"Apple præsenterer i morgen den næste iPhone, t...",2017-09-11,N/A,ansigtsscan,Ny mobilmode: Nu bliver telefonerne rammeløse ...
4,https://www.dr.dk/nyheder/viden/tech/opsamling...,Opsamling: Anmeldere tager pænt imod iPhone X,"Der er store roser til Apples nye, hundedyre i...","“9 ud af 10”, “1.000 dollars værd”, “nem at an...",2017-11-06,N/A,ansigtsscan,Opsamling: Anmeldere tager pænt imod iPhone X ...
5,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet 'råber vagt i gevær': Her er spør...,,I sidste uge fik politiet grønt lys til brug a...,2024-09-14,indland,masseovervågning ansigt,Datatilsynet 'råber vagt i gevær': Her er spør...
6,https://www.dr.dk/nyheder/udland/her-er-fire-t...,"Her er fire teknologier, der holder kinesiske ...",,Politifolk har stormet kinesiske gader og angi...,2022-11-29,udland,masseovervågning ansigt,"Her er fire teknologier, der holder kinesiske ..."
7,https://www.dr.dk/nyheder/udland/it-gigant-skr...,It-gigant skrotter sin egen teknologi og advar...,Myndigheder bør være påpasselige med at bruge ...,It-giganten IBM har besluttet at droppe udvikl...,2020-06-09,udland,masseovervågning ansigt,It-gigant skrotter sin egen teknologi og advar...
8,https://www.dr.dk/nyheder/viden/tech/wikileaks...,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...,Analyse: Lækkede CIA-dokumenter fortæller om t...,Den 7. marts blev en stor samling dokumenter m...,2017-03-21,N/A,masseovervågning ansigt,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...
9,https://www.dr.dk/nyheder/politik/sf-vil-have-...,SF vil have kommission til at kulegrave overvå...,Alle partier ønsker mere viden om overvågninge...,SF vil have regeringen til at nedsætte en komm...,2014-04-25,politik,masseovervågning ansigt,SF vil have kommission til at kulegrave overvå...


In [ ]:
df = df[df['merged_content'].str.strip() != 'no subheader']
df

,url,header,subheader,content,date,category,search term,merged_content
0,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet 'råber vagt i gevær': Her er spør...,NaN,I sidste uge fik politiet grønt lys til brug a...,2024-09-14,indland,ansigtsgenkendelse,Datatilsynet 'råber vagt i gevær': Her er spør...
2,https://www.dr.dk/nyheder/indland/fodboldklubb...,Fodboldklubber vil have ansigtsgenkendelse på ...,Et nationalt register for hooligans var øverst...,"Det er rockere og banderelaterede hooligans, d...",2024-09-12,indland,ansigtsgenkendelse,Fodboldklubber vil have ansigtsgenkendelse på ...
3,https://www.dr.dk/nyheder/indland/justitsminis...,Justitsminister: FCK-Brøndby-kampe uden tilsku...,"Vold og ballade skal lukkes helt ned, når de t...","Gabende tomme tribuner uden råb, sange og tils...",2024-09-12,indland,ansigtsgenkendelse,Justitsminister: FCK-Brøndby-kampe uden tilsku...
4,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet har sendt række af spørgsmål til ...,Politiet har fra mandag tre uger til at besvar...,Datatilsynet har sendt en række spørgsmål til ...,2024-09-09,indland,ansigtsgenkendelse,Datatilsynet har sendt række af spørgsmål til ...
5,https://www.dr.dk/nyheder/indland/hvor-meget-m...,Hvor meget må din nye arbejdsgiver vide? Tidli...,Den tidligere PET-chef Jakob Scharf hjælper vi...,"Er der noget, du har glemt at fortælle din nye...",2024-09-06,indland,ansigtsgenkendelse,Hvor meget må din nye arbejdsgiver vide? Tidli...
...,...,...,...,...,...,...,...,...
151,https://www.dr.dk/nyheder/regionale/hovedstads...,Røver meldte sig til politiet,"I aftes meldte en mand, der mistænkes for et r...",En ekspedient i en 7-eleven på en Shell-tank p...,2012-06-26,hovedstadsomraadet,videoovervågningskamera,Røver meldte sig til politiet I aftes meldte e...
152,https://www.dr.dk/nyheder/indland/oeltyv-faeld...,Øltyv fældet af høm høm,En tyv fra Silkeborg kom i dén grad til at afs...,I nat blev en rigtig lortenat for en 16-årig d...,2011-01-05,indland,videoovervågningskamera,Øltyv fældet af høm høm En tyv fra Silkeborg k...
153,https://www.dr.dk/nyheder/indland/politiet-off...,Politiet offentliggør billeder af brandmistænkte,Københavns Politi offentliggør videooptagelser...,"Københavns Politi mener, at man er tæt på en o...",2008-02-21,indland,videoovervågningskamera,Politiet offentliggør billeder af brandmistænk...
154,https://www.dr.dk/nyheder/regionale/hovedstads...,Kommuner overvåger skoler og institutioner med...,NaN,"Af artiklen fremgik det, at DR har sendt en kr...",2021-07-09,hovedstadsomraadet,videoovervågningskamera,Kommuner overvåger skoler og institutioner med...


In [ ]:
from google.colab import files

df.to_csv('df_DR_all_terms_clean.csv')
files.download('df_DR_all_terms_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/DR/df_DR_all_terms_clean (1).csv')
df1 = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/DR/df_DR_all_terms_clean.csv')

In [ ]:
df = pd.concat([df, df1])
df.drop(columns=['Unnamed: 0'], inplace=True)

,url,header,subheader,content,date,category,search term,merged_content
0,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebooks ansigtsgenkendelse er muligvis ulovlig,Facebook er begyndt at analysere ansigter på e...,"Inden længe vil Facebook spørge dig, om de må ...",2018-04-20,teknologi,face recognition,Facebooks ansigtsgenkendelse er muligvis ulovl...
1,https://www.dr.dk/nyheder/viden/teknologi/face...,Facebook vil have dig til at sige ‘ja’ til ans...,Facebook genintroducerer kontroversiel ansigts...,Må Facebook have lov til at bruge en teknologi...,2018-04-18,teknologi,face recognition,Facebook vil have dig til at sige ‘ja’ til ans...
2,https://www.dr.dk/nyheder/viden/tech/senator-h...,Senator: Hvor sikker er Apples nye ansigtsgenk...,"Det amerikanske senat vil have svar på, hvor s...","Da Apple tirsdag lancerede sin nye iPhone X, v...",2017-09-14,NaN,ansigtsscan,Senator: Hvor sikker er Apples nye ansigtsgenk...
3,https://www.dr.dk/nyheder/viden/tech/ny-mobilm...,Ny mobilmode: Nu bliver telefonerne rammeløse,De nye telefoner ligner næsten bare en glaspla...,"Apple præsenterer i morgen den næste iPhone, t...",2017-09-11,NaN,ansigtsscan,Ny mobilmode: Nu bliver telefonerne rammeløse ...
4,https://www.dr.dk/nyheder/viden/tech/opsamling...,Opsamling: Anmeldere tager pænt imod iPhone X,"Der er store roser til Apples nye, hundedyre i...","“9 ud af 10”, “1.000 dollars værd”, “nem at an...",2017-11-06,NaN,ansigtsscan,Opsamling: Anmeldere tager pænt imod iPhone X ...
5,https://www.dr.dk/nyheder/indland/datatilsynet...,Datatilsynet 'råber vagt i gevær': Her er spør...,NaN,I sidste uge fik politiet grønt lys til brug a...,2024-09-14,indland,masseovervågning ansigt,Datatilsynet 'råber vagt i gevær': Her er spør...
6,https://www.dr.dk/nyheder/udland/her-er-fire-t...,"Her er fire teknologier, der holder kinesiske ...",NaN,Politifolk har stormet kinesiske gader og angi...,2022-11-29,udland,masseovervågning ansigt,"Her er fire teknologier, der holder kinesiske ..."
7,https://www.dr.dk/nyheder/udland/it-gigant-skr...,It-gigant skrotter sin egen teknologi og advar...,Myndigheder bør være påpasselige med at bruge ...,It-giganten IBM har besluttet at droppe udvikl...,2020-06-09,udland,masseovervågning ansigt,It-gigant skrotter sin egen teknologi og advar...
8,https://www.dr.dk/nyheder/viden/tech/wikileaks...,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...,Analyse: Lækkede CIA-dokumenter fortæller om t...,Den 7. marts blev en stor samling dokumenter m...,2017-03-21,NaN,masseovervågning ansigt,WikiLeaks-lækagen: Sådan fungerer CIAs værktøj...
9,https://www.dr.dk/nyheder/politik/sf-vil-have-...,SF vil have kommission til at kulegrave overvå...,Alle partier ønsker mere viden om overvågninge...,SF vil have regeringen til at nedsætte en komm...,2014-04-25,politik,masseovervågning ansigt,SF vil have kommission til at kulegrave overvå...


In [ ]:
from google.colab import files

df.to_csv('DR.csv')
files.download('DR.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>